## 0. Cargando la Data y Packages

In [16]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("StatsBase")



   Resolving package versions...
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
   Installed IrrationalConstants ─ v0.2.2
   Installed LogExpFunctions ───── v0.3.28
   Installed DocStringExtensions ─ v0.9.3
   Installed StatsAPI ──────────── v1.7.0
   Installed StatsBase ─────────── v0.34.3
    Updating `C:\Users\abrah\.julia\environments\v1.10\Project.toml`
  [2913bbd2] + StatsBase v0.34.3
    Updating `C:\Users\abrah\.julia\environments\v1.10\Manifest.toml`
  [ffbed154] + DocStringExtensions v0.9.3
  [92d709cd] + IrrationalConstants v0.2.2
  [2ab3a3ac] + LogExpFunctions v0.3.28
  [82ae8749] + StatsAPI v1.7.0
  [2913bbd2] + StatsBase v0.34.3
  [76f85450] + LibGit2
  [ca575930] + Ne

In [17]:
using CSV
using DataFrames

df = CSV.read("../../../data/wage2015_subsample_inference.csv", DataFrame)

#para ver las primeras 5 observacione

first(df, 5) 


Row,rownames,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,10,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,12,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,15,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,18,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,19,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22


## 1. Data analysis

#### 2. Are there missing values? Display the number of missings (if any) of each variable in the data set.


#### 3. Report descriptive statistics of the variables (mean, standard deviation, percentiles, etc.). Interpret your results.

In [18]:
# Usamos describe para obtener algunos estadísticos descriptivos y ver los missingvalues de caca variable del data set
describe(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,rownames,15636.3,10,15260.0,32643,0,Int64
2,wage,23.4104,3.02198,19.2308,528.846,0,Float64
3,lwage,2.97079,1.10591,2.95651,6.2707,0,Float64
4,sex,0.444466,0.0,0.0,1.0,0,Float64
5,shs,0.023301,0.0,0.0,1.0,0,Float64
6,hsg,0.243883,0.0,0.0,1.0,0,Float64
7,scl,0.278058,0.0,0.0,1.0,0,Float64
8,clg,0.31767,0.0,0.0,1.0,0,Float64
9,ad,0.137087,0.0,0.0,1.0,0,Float64


In [21]:
#encontramos algunos los cuartiles de la variable wage:
q1 = quantile(df.wage, 0.25)  
q2 = quantile(df.wage, 0.50)  
q3 = quantile(df.wage, 0.75)  

#imprimimos los valores de cada cuartil
println("Q1: $q1")
println("Mediana: $q2")
println("Q3: $q3")


Q1: 13.461538461538462
Mediana: 19.23076923076923
Q3: 27.77777777777778


### 4. How many women with a college graduate degree (clg) or above have a wage corresponding to the 25% richest of the sample? Report the dataframe corresponding to this conditions and the result

In [32]:
#nos están preguntando por cuantas muejeres graduadas de universidad o con postgrado se encuentran el cuartil superior de salarios. (mayores a 27.7777)

df[(df.sex .== 1.0) .& ((df.clg .== 1.0) .|| (df.ad .== 1.0)) .& (df.wage .>= q3), :]

Row,rownames,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,19,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
2,191,42.3077,3.74497,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4700.0,16,4795.0,9
3,232,41.2088,3.71865,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,11.0,1.21,1.331,1.4641,4700.0,16,5790.0,9
4,319,100.0,4.60517,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,27.0,7.29,19.683,53.1441,850.0,2,6970.0,12
5,563,33.6538,3.51613,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0,0.16,0.064,0.0256,710.0,2,7390.0,14
6,842,36.0577,3.58512,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,23.0,5.29,12.167,27.9841,420.0,1,9170.0,21
7,865,31.25,3.44202,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,15.0,2.25,3.375,5.0625,3255.0,10,8190.0,18
8,892,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,9.0,0.81,0.729,0.6561,540.0,2,6990.0,12
9,1052,48.0769,3.8728,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,24.0,5.76,13.824,33.1776,4850.0,16,3390.0,6


In [37]:
#para saber el número de mujeres que cumplen con estos requisitos:
mujeres_with_college_or_superior_degree_in_q3=nrow(df[(df.sex .== 1.0) .& ((df.clg .== 1.0) .|| (df.ad .== 1.0)) .& (df.wage .>= q3), :])

println("Las mujeres con universidad o un grado superior en el cuartil superior de salios son: $mujeres_with_college_or_superior_degree_in_q3")

Las mujeres con universidad o un grado superior en el cuartil superior de salios son: 419


#### 5. How many men with a high school graduate degree (hsg) or below have a wage corresponding to the 25% richest of the sample? Report the dataframe corresponding to this conditions and the result.

In [39]:
#hacemos algo similar para los hombres:
men_with_highschool_or_no_degree_in_q3=nrow(df[(df.sex .== 0.0) .& ((df.shs .== 1.0) .|| (df.hsg .== 1.0)) .& (df.wage .>= q3), :])

println("Los hombres con high school terminada o un grado menor en el cuartil superior de salios son: $men_with_highschool_or_no_degree_in_q3")

Los hombres con high school terminada o un grado menor en el cuartil superior de salios son: 118


#### 6. Create two dataframes. One containing only the log(wage) and the other containig every variable of the data set but the wage related variables.

In [45]:
#creamos los 2 dataframes

#1er dataframe (only log(wage))

only_logwage = DataFrame(log_wage = df.lwage)

#2do dataframe (otras variables)

df_no_wage = select(df, Not([:wage, :lwage]))



Row,rownames,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,10,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,12,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,15,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,18,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,19,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
6,30,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,1650.0,5,7460.0,14
7,43,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,5120.0,17,7280.0,14
8,44,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,5240.0,17,5680.0,9
9,47,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,4040.0,13,8590.0,19


## 2. Data wrangling

#### 7. Make an array for our Y variable, which will be the logarithm of wage (lwage column)

In [46]:
#convierto en array con nombre Y
Y=df.lwage

5150-element Vector{Float64}:
 2.2633643798407643
 3.872802292274865
 2.403126322215923
 2.634927936273247
 3.361976668508874
 2.4622152385859297
 2.9565115604007097
 2.9565115604007097
 2.4849066497880004
 2.9565115604007097
 ⋮
 3.117779707996832
 2.822980167776187
 3.1796551117149194
 2.6280074934286737
 2.6925460145662448
 3.138833117194664
 3.649658740960655
 3.4955080611333966
 2.8511510447428834

## 3. Linear Regressions